In [ ]:
!wget https://www.dropbox.com/s/n30i1tiwi3s64fg/X-Ray_B.zip?dl=0

In [ ]:
!unzip X-Ray_B.zip?dl=0

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50,VGG16,MobileNet,DenseNet121
from tensorflow.keras.applications.resnet50 import preprocess_input

In [4]:
train_dir = "X-Ray/train/"
valid_dir = "X-Ray/test/"

img_width, img_height = 224, 224

##DenseNet121

In [5]:
conv_base = DenseNet121(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))

conv_base.summary()

29097984/29084464 [==============================] - 0s 0us/step
Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                       

## Features Extraction

In [6]:
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 1024))  
    # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count,))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='binary')
    
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(train_dir, 1046)  
validation_features, validation_labels = extract_features(valid_dir, 466)

Found 1046 images belonging to 2 classes.
Found 466 images belonging to 2 classes.


In [7]:
train_labels

array([1., 0., 0., ..., 0., 1., 1.])

In [8]:
nsamples1, nx1, ny1 ,nz1= train_features.shape
d2_train_dataset = train_features.reshape((nsamples1,nx1*ny1*nz1))

In [9]:
nsamples, nx, ny ,nz= validation_features.shape
d2_test_dataset = validation_features.reshape((nsamples,nx*ny*nz))

### Implementing SVM

In [10]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svc=SVC() 
svc.fit(d2_train_dataset,train_labels)

y_pred=svc.predict(d2_test_dataset)

In [12]:
#print('Model accuracy score with default hyperparameters: {0:0.4f}' . format(accuracy_score(validation_labels, y_pred)))
print('Model accuracy score with default hyperparameters: {0:0.4f} : ')

Model accuracy score with default hyperparameters: {0:0.4f} : 


In [13]:
print('Model accuracy score with default hyperparameters: {0:0.4f}' . format(accuracy_score(validation_labels, y_pred)))

Model accuracy score with default hyperparameters: 0.9764


In [14]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98       230
         1.0       0.98      0.97      0.98       236

    accuracy                           0.98       466
   macro avg       0.98      0.98      0.98       466
weighted avg       0.98      0.98      0.98       466



In [15]:
confusion_matrix(y_pred,validation_labels)

array([[226,   4],
       [  7, 229]])

### Implementing a Random Forrest

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
newmodel=RandomForestClassifier()

In [18]:
newmodel.fit(d2_train_dataset,train_labels)

RandomForestClassifier()

In [19]:
y_pred=newmodel.predict(d2_test_dataset)
y_pred

array([1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 0., 1.

In [20]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.95      0.98      0.97       227
         1.0       0.98      0.95      0.97       239

    accuracy                           0.97       466
   macro avg       0.97      0.97      0.97       466
weighted avg       0.97      0.97      0.97       466



In [21]:
confusion_matrix(y_pred,validation_labels)

array([[222,   5],
       [ 11, 228]])

### Implementing a KNN

In [22]:
from sklearn.neighbors import KNeighborsClassifier

In [23]:
knn=KNeighborsClassifier(n_neighbors=7)

In [24]:
knn.fit(d2_train_dataset,train_labels)

KNeighborsClassifier(n_neighbors=7)

In [25]:
y_pred_knn=knn.predict(d2_test_dataset)
y_pred_knn

array([1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
       0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 0., 1., 0., 1.

In [26]:
accuracy_score(y_pred_knn,validation_labels)
print(classification_report(y_pred_knn,validation_labels))

              precision    recall  f1-score   support

         0.0       0.91      0.95      0.93       223
         1.0       0.95      0.91      0.93       243

    accuracy                           0.93       466
   macro avg       0.93      0.93      0.93       466
weighted avg       0.93      0.93      0.93       466



In [27]:
confusion_matrix(y_pred_knn,validation_labels)

array([[211,  12],
       [ 22, 221]])

## Implementing a Decision Tree

In [28]:
from sklearn.tree import DecisionTreeClassifier

In [29]:
dtc=DecisionTreeClassifier()

In [30]:
dtc.fit(d2_train_dataset,train_labels)

DecisionTreeClassifier()

In [31]:
y_pred_dtc=dtc.predict(d2_test_dataset)

In [32]:
y_pred_dtc

array([1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 1.

In [33]:
accuracy_score(y_pred_dtc,validation_labels)
print(classification_report(y_pred_dtc,validation_labels))

              precision    recall  f1-score   support

         0.0       0.89      0.81      0.85       257
         1.0       0.79      0.88      0.83       209

    accuracy                           0.84       466
   macro avg       0.84      0.84      0.84       466
weighted avg       0.85      0.84      0.84       466



In [34]:
confusion_matrix(y_pred_dtc,validation_labels)

array([[208,  49],
       [ 25, 184]])

## Implementing Gaussian Naive Bayes

In [35]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(d2_train_dataset,train_labels)

GaussianNB()

In [36]:
y_pred=dtc.predict(d2_test_dataset)

In [37]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.89      0.81      0.85       257
         1.0       0.79      0.88      0.83       209

    accuracy                           0.84       466
   macro avg       0.84      0.84      0.84       466
weighted avg       0.85      0.84      0.84       466



## Implementing AdaBoost

In [38]:
from sklearn.ensemble import AdaBoostClassifier

In [39]:
classifier = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200
)
classifier.fit(d2_train_dataset, train_labels)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   n_estimators=200)

In [40]:
predictions = classifier.predict(d2_test_dataset)

In [41]:
accuracy_score(predictions,validation_labels)
print(classification_report(predictions,validation_labels))

              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96       233
         1.0       0.96      0.96      0.96       233

    accuracy                           0.96       466
   macro avg       0.96      0.96      0.96       466
weighted avg       0.96      0.96      0.96       466

